<a href="https://colab.research.google.com/github/srohit310/scomp_project/blob/main/yt_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytube
!pip install imageio==2.4.1
!pip install youtube_comment_downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 21.2 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=3e9731c6f48a95055da845d1270fd972b9ab48d6e91ae25deb0aa6bc6c5a2280
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 20.4 MB/s 


In [2]:
from pytube import YouTube, extract
from pytube.cli import on_progress
from moviepy.editor import *
import os, uuid, cv2, pandas as pd
from itertools import islice
from youtube_comment_downloader import *
from IPython.display import Markdown, display

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)4087808/45929032 bytes (8.9%)8380416/45929032 bytes (18.2%)12615680/45929032 bytes (27.5%)16850944/45929032 bytes (36.7%)21176320/45929032 bytes (46.1%)24444928/45929032 bytes (53.2%)27639808/45929032 bytes (60.2%)31301632/45929032 bytes (68.2%)35356672/45929032 bytes (77.0%)39649280/45929032 bytes (86.3%)43778048/45929032 bytes (95.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [3]:
def printmd(string):
    display(Markdown(string))

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('drive/MyDrive')

if not os.path.exists('scomp_project'):
    os.mkdir('scomp_project')
    printmd("**A folder called scomp_project has been made for you, Please insert the video data csv file into it and rename it to video_data_url.csv**")

os.chdir('scomp_project')

Mounted at /content/drive/


In [5]:
def fetch_comments(link):

    comments = []

    while True:

        try:
            downloader = YoutubeCommentDownloader()
            comments_list = []
            comments = downloader.get_comments_from_url(link, sort_by=SORT_BY_POPULAR)
            break
        
        except:
            continue

    for comment in islice(comments, 10):
        comments_list.append(comment['text'])
    
    return comments_list

In [6]:
def DownloadYoutube(link, id, new_df):

    name = ''

    try:
        youtubeObject = YouTube(link, on_progress_callback=on_progress)

        new_df[id]['title'] = youtubeObject.title
        new_df[id]['description'] = youtubeObject.description
        new_df[id]['comments'] = fetch_comments(link)
        new_df[id]['filenames'] = []

        youtubeObject = youtubeObject.streams.get_highest_resolution()

        name = str(uuid.uuid4()) + '.mp4'
        youtubeObject.download(output_path='videos',filename=name)
        print("Download is completed successfully")

    except:
        name = ''
        print("An error has occurred")

    return name, new_df

In [7]:
def convert_to_seconds(time):

    time_div = time.split(':')
    m_arr = [1,60,60*60]
    seconds = 0

    for count, time in enumerate(reversed(time_div)):
        seconds += int(time)*m_arr[count]    

    return seconds

In [11]:
def collect_images(path ,img_nos):

    cam = cv2.VideoCapture(path+".mp4")
    totframe = 0
  
    while(True):
      
        ret, frame = cam.read()
        if ret: totframe += 1
        else: break

    cam.release()
    cam = cv2.VideoCapture(path+".mp4")

    curr_frame = 0
    capt_inc = int(totframe/(img_nos+1))
    capt_pnt = int(totframe/(img_nos+1))
    orig_img_nos = img_nos

    if not os.path.exists(path):
        os.mkdir(path)

    while img_nos != 0:
      
        ret, frame = cam.read()
        if ret:

            if curr_frame == capt_pnt:
                
                name = path +'/frame'+ str(curr_frame) + '.jpg'
                cv2.imwrite(name, frame)
                capt_pnt += capt_inc
                img_nos -= 1
                
            curr_frame += 1

        else:
            break
    
    cam.release()
    cv2.destroyAllWindows()

    print('Collected '+str(orig_img_nos)+' images from video\n\n')

In [12]:
def crop_out_video(filename, timefetch, emotion, id, new_df, index):

    timelimits = timefetch.split('-')
    start_time = timelimits[0].strip()
    end_time = timelimits[1].strip()
            
    start_time = convert_to_seconds(start_time)
    end_time = convert_to_seconds(end_time)
            
    video = VideoFileClip('videos/'+filename).subclip(start_time, end_time)

    if not os.path.exists('cropped_videos/'+emotion):
        os.mkdir('cropped_videos/'+emotion)
            
    new_filename = str(uuid.uuid4())
    new_df[id]['filenames'].append(new_filename+ '.mp4')
    video.write_videofile('cropped_videos/'+emotion+'/'+new_filename+ '.mp4') 
    video.close()
    print('Video '+str(index)+' downloaded')

    collect_images('cropped_videos/'+emotion+'/'+new_filename, 6)

    return new_df

In [13]:
if __name__ == '__main__':

    df = pd.read_csv('video_data_url.csv')

    new_df = {}

    link_y_map = {}

    if not os.path.exists('videos'):
        os.mkdir('videos')

    if not os.path.exists('cropped_videos'):
        os.mkdir('cropped_videos')
    
    for index, row in df.iterrows():
        
        link = row['Url']

        id = extract.video_id(link)

        if id in link_y_map:
            filename = link_y_map[id]
        else:
            new_df[id] = {}
            new_df[id]['emotion'] = row['Emotion']
            filename, new_df = DownloadYoutube(link, id, new_df)
            link_y_map[id] = filename
    
        if len(filename)>0:

            new_df = crop_out_video(filename, row['Timestamp'], row['Emotion'], id, new_df, index)

    post_df = pd.DataFrame.from_dict(new_df)
    post_df = post_df.transpose()
    post_df.to_csv('cropped_videos/posts_data.csv', encoding='utf-8-sig')

Download is completed successfully
[MoviePy] >>>> Building video cropped_videos/Happy/56382254-07b5-41a9-8a47-1053748c0567.mp4
[MoviePy] Writing audio in 56382254-07b5-41a9-8a47-1053748c0567TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:00<00:00, 679.20it/s]

[MoviePy] Done.
[MoviePy] Writing video cropped_videos/Happy/56382254-07b5-41a9-8a47-1053748c0567.mp4



100%|█████████▉| 300/301 [00:11<00:00, 25.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cropped_videos/Happy/56382254-07b5-41a9-8a47-1053748c0567.mp4 

Video 0 downloaded
Collected 6 images from video


[MoviePy] >>>> Building video cropped_videos/Happy/e53bd6ef-7fbc-4fe2-8fbd-f6fb5b6477b2.mp4
[MoviePy] Writing audio in e53bd6ef-7fbc-4fe2-8fbd-f6fb5b6477b2TEMP_MPY_wvf_snd.mp3


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: cropped_videos/Happy/04710b98-12d2-492d-9c0d-5f112529fbb3.mp4 

Video 74 downloaded
Collected 6 images from video


[MoviePy] >>>> Building video cropped_videos/Happy/e16040ed-1ea2-4cc4-a8af-0beef02188e6.mp4
[MoviePy] Writing audio in e16040ed-1ea2-4cc4-a8af-0beef02188e6TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:00<00:00, 428.48it/s]

[MoviePy] Done.
[MoviePy] Writing video cropped_videos/Happy/e16040ed-1ea2-4cc4-a8af-0beef02188e6.mp4



100%|█████████▉| 240/241 [00:08<00:00, 26.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cropped_videos/Happy/e16040ed-1ea2-4cc4-a8af-0beef02188e6.mp4 

Video 75 downloaded
Collected 6 images from video


